In [1]:
import json

In [2]:
with open('./profile-www.cryptonoter.me.json') as f:
    raw = json.load(f)

In [3]:
profile = raw['profile']
profile.keys()

dict_keys(['nodes', 'samples', 'startTime', 'endTime', 'timeDeltas'])

In [4]:
import numpy as np

# Unit: microseconds
np.average(profile['timeDeltas']), np.std(profile['timeDeltas'])

(198.75716908608186, 318.72477710263485)

In [5]:
# A list of all function names
set(x['callFrame']['functionName'] for x in profile['nodes'])

{'',
 '$a',
 '$c.encode',
 '$d',
 '(garbage collector)',
 '(idle)',
 '(program)',
 '(root)',
 'Ac',
 'Add',
 'Ba',
 'Bc',
 'Bd',
 'C',
 'Ca',
 'CallProperty1',
 'CallUndefinedReceiver1',
 'ConvertToString',
 'Da.add',
 'Da.addAll',
 'DoJoin',
 'E',
 'Ec',
 'Fd',
 'G',
 'Gd',
 'Ha.D',
 'Hd',
 'Hd.a.push',
 'Inc',
 'InnerArrayJoin',
 'InvokeIntrinsic',
 'J',
 'Jc',
 'Jd',
 'Join',
 'Jump',
 'JumpLoop',
 'Kc',
 'LdaConstant',
 'LdaGlobal',
 'LdaImmutableContextSlot',
 'LdaKeyedProperty',
 'LdaNamedProperty',
 'LdaUndefined',
 'Ldar',
 'Ma',
 'Md',
 'Mod',
 'Mov',
 'N.N',
 'N.create',
 'ObjectPrototypeHasOwnProperty',
 'Pa',
 'Pd',
 'Qd',
 'RegExp: \\b\\w+\\b',
 'RegExpPrototypeExec',
 'RegExpPrototypeTest',
 'Return',
 'S',
 'Sa',
 'Sb',
 'Sd',
 'StaKeyedProperty',
 'StackCheck',
 'StringPrototypeCharCodeAt',
 'StringPrototypeMatch',
 'StringPrototypeReplace',
 'StringPrototypeSlice',
 'SubSmi',
 'T',
 'Tb',
 'Tc',
 'Td',
 'TestEqualStrict',
 'ToBooleanLogicalNot',
 'Ua',
 'Ub',
 'Ud',
 '

In [6]:
# This should be equivalent to "sorting by self time" in chrome gui
sorted(profile['nodes'], key=lambda x: x['hitCount'], reverse=True)[:4]

[{'callFrame': {'columnNumber': -1,
   'functionName': '(idle)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 299699,
  'id': 4},
 {'callFrame': {'columnNumber': -1,
   'functionName': '(program)',
   'lineNumber': -1,
   'scriptId': '0',
   'url': ''},
  'hitCount': 2520,
  'id': 2},
 {'callFrame': {'columnNumber': 29277,
   'functionName': '',
   'lineNumber': 0,
   'scriptId': '19',
   'url': ''},
  'children': [278, 280, 285, 289, 295, 301, 315, 316],
  'hitCount': 123,
  'id': 272,
  'positionTicks': [{'line': 1, 'ticks': 123}]},
 {'callFrame': {'columnNumber': 25413,
   'functionName': '_0x2d0c.lbeIEt',
   'lineNumber': 0,
   'scriptId': '16',
   'url': 'https://www.cryptonoter.me/prototyper-show.js'},
  'children': [11, 12, 13, 14, 15, 16, 17, 18, 19],
  'hitCount': 63,
  'id': 10,
  'positionTicks': [{'line': 1, 'ticks': 63}]}]

In [7]:
# This should be equivalent to "Call tree" in chrome gui

result = {}
def calc(node_id):
    cached = result.get(node_id)
    if cached:
        return cached
    
    node = profile['nodes'][node_id - 1]
    child = node.get('children', [])
    ans = node['hitCount']
    for cid in child:
        ans += calc(cid)
    
    result[node_id] = ans
    return ans

calc(1)
sorted([(k,v,profile['nodes'][k-1])for k,v in result.items()], key=lambda x: x[1], reverse=True)[:50]

[(1,
  302842,
  {'callFrame': {'columnNumber': -1,
    'functionName': '(root)',
    'lineNumber': -1,
    'scriptId': '0',
    'url': ''},
   'children': [2,
    3,
    4,
    5,
    48,
    49,
    76,
    123,
    134,
    140,
    245,
    262,
    264,
    265,
    272,
    273,
    276,
    277,
    283],
   'hitCount': 0,
   'id': 1}),
 (4,
  299699,
  {'callFrame': {'columnNumber': -1,
    'functionName': '(idle)',
    'lineNumber': -1,
    'scriptId': '0',
    'url': ''},
   'hitCount': 299699,
   'id': 4}),
 (2,
  2520,
  {'callFrame': {'columnNumber': -1,
    'functionName': '(program)',
    'lineNumber': -1,
    'scriptId': '0',
    'url': ''},
   'hitCount': 2520,
   'id': 2}),
 (5,
  147,
  {'callFrame': {'columnNumber': 0,
    'functionName': '',
    'lineNumber': 0,
    'scriptId': '16',
    'url': 'https://www.cryptonoter.me/prototyper-show.js'},
   'children': [6, 8, 20, 30],
   'hitCount': 12,
   'id': 5,
   'positionTicks': [{'line': 1, 'ticks': 12}]}),
 (273,
  14

limit_output extension: Maximum message size of 5000 exceeded with 14434 characters

In [8]:
# Script id -> URLs
dict([
    (node['callFrame']['scriptId'], node['callFrame']['url'])
    for node in profile['nodes']
])

{'0': '',
 '16': 'https://www.cryptonoter.me/prototyper-show.js',
 '19': '',
 '22': 'https://www.cryptonoter.me/demo.php',
 '23': 'https://www.googletagmanager.com/gtag/js?id=UA-112101534-1',
 '24': 'https://www.google-analytics.com/analytics.js',
 '4': 'native array.js'}

In [9]:
# Now we suspect Node 273, 5...
# What are they doing?
def getAllChildren(cid):
    node = profile['nodes'][cid-1]
    child = node.get('children', [])
    ans = list(child)
    for x in child:
        ans += getAllChildren(x)
    return ans

print(max([(result[x], profile['nodes'][x-1]['callFrame']) for x in getAllChildren(273)], key=lambda x: x[0]))
print(max([(result[x], profile['nodes'][x-1]['callFrame']) for x in getAllChildren(5)], key=lambda x: x[0]))

(8, {'scriptId': '19', 'url': '', 'lineNumber': 0, 'functionName': 'h.(anonymous function).(anonymous function)', 'columnNumber': 16161})
(99, {'scriptId': '16', 'url': 'https://www.cryptonoter.me/prototyper-show.js', 'lineNumber': 0, 'functionName': '_0x2d0c', 'columnNumber': 24543})


In [10]:
# But what's their actual runtime? We only know hit counts.
# Estimate through profile['timeDeltas']:
child = getAllChildren(273) + [273]
print(child)
sum([profile['timeDeltas'][i] for i, x in enumerate(profile['samples']) if x in child])

[278, 280, 285, 289, 295, 301, 315, 316, 286, 287, 312, 314, 320, 322, 293, 275, 326, 273]


8591

In [11]:
# How much time is this, compared to "(root) - (idle)" ?
root = profile['nodes'][0]
idle = profile['nodes'][3]
assert root['callFrame']['functionName'] == '(root)'
assert idle['callFrame']['functionName'] == '(idle)'
print((result[273] + result[5]) / (result[root['id']] - result[idle['id']]))

0.0922685332484887
